# 9.2 첫 번째 계산 그래프를 만들어 세션에서 실행하기

In [1]:
# 아래의 코드는 계산 그래프만 만들뿐 실제 평가는 세션을 시작하고 변수를 초기화한 후 평가해야만 한다.
import tensorflow as tf

x = tf.Variable(3, name="x")
y = tf.Variable(4, name="y")
f = x*x*y + y + 2


ModuleNotFoundError: No module named 'tensorflow'

## 세션을 만드는 여러가지 방법

In [3]:
sess = tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
result = sess.run(f)
print(result)

sess.close()

42


In [5]:
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result = f.eval()
    print(result)

42


In [8]:
init = tf.global_variables_initializer()
with tf.Session() as sess:
    init.run()
    result = f.eval()
    print(result)

42


In [9]:
sess = tf.InteractiveSession()
init.run()
result = f.eval()
print(result)
sess.close()

42


# 9.3 계산 그래프 관리

In [16]:
x1 = tf.Variable(1)
x1.graph is tf.get_default_graph()

True

In [19]:
# 독립적인 계산 그래프를 만들어야 할때, 새로운 Graph 객체를 만들어 with 블록 안에서 임시로 기본 계산 그래프로 사용
graph = tf.Graph()
with graph.as_default():
    x2 = tf.Variable(2)
print(x2.graph is graph)
print(x2.graph is tf.get_default_graph())

True
False


# 9.4 노드 값의 생애주기

In [22]:
# 아래 코드에서 y 평가시 y 가 x에 의존한다는 것을 판단하고 x 를 먼저 평가한다.
# 주의! y, z 두곳에서 x를 참조하더라도 한번 계산된 x를 재활용하지는 않는다.
w = tf.constant(3)
x = w + 2
y = x + 5
z = x * 3

with tf.Session() as sess:
    print(y.eval())
    print(z.eval())

10
15


In [25]:
# 모든 노드의 값은 계산 그래프 실행 간에 유지되지 않는다.
# 유지하기 위해서는 한개 그래프 실행시 y, z를 모두 평가하도록 해야한다.
with tf.Session() as sess:
    y_val, z_val = sess.run([y, z])
    print(y_val)
    print(z_val)


10
15


# 9.5 텐서플로를 이용한 선형 회귀
* 텐서플로 연산(ops) : 여러 개의 입력을 받아 출력을 만듬
* 소스 연산(source ops) : 입력이 없는 상수와 변수 연산
* 텐서(tensor) : 입력과 출력의 데이터 형식이며 다차원 배열이다. 넘파이 배열과 비슷하게 타입과 크기를 갖는다.

In [32]:
# 2장에서 다뤘던 캘리포니아 주택 가격 데이터셋에 선형회귀 수행
import numpy as np
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
m, n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]

X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
XT = tf.transpose(x)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), y)

with tf.Session() as sess:
    theta_value = theta.eval()
    print(theta_value)

[[-3.7185181e+01]
 [ 4.3633747e-01]
 [ 9.3952334e-03]
 [-1.0711310e-01]
 [ 6.4479220e-01]
 [-4.0338000e-06]
 [-3.7813708e-03]
 [-4.2348403e-01]
 [-4.3721911e-01]]


# 9.6 경사 하강법 구현
* 정규방정식(OLS) 대신 경사하강법 사용
* 경사 하강법 사용시에는 입력 특성 벡터를 정규화하지 않을 경우 훈련 속도가 매우 느려질 수 있음.
* <font color='red'>TODO</font> 그래디언트디센트 스터디하기

## 9.6.1 직접 그래디언트 계산
* random_uniform() : 난수를 담은 텐서를 생성하는 노드를 그래프에 생성
* assign() : 변수에 새로운 값을 할당하는 노드 생성

In [46]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_housing_data = scaler.fit_transform(housing.data)
scaled_housing_data_plus_bias = np.c_[np.ones((m, 1)), scaled_housing_data]

print(scaled_housing_data_plus_bias.mean(axis=0))
print(scaled_housing_data_plus_bias.mean(axis=1))
print(scaled_housing_data_plus_bias.mean())
print(scaled_housing_data_plus_bias.shape)

[ 1.00000000e+00  6.60969987e-17  5.50808322e-18  6.60969987e-17
 -1.06030602e-16 -1.10161664e-17  3.44255201e-18 -1.07958431e-15
 -8.52651283e-15]
[ 0.38915536  0.36424355  0.5116157  ... -0.06612179 -0.06360587
  0.01359031]
0.11111111111111005
(20640, 9)


In [47]:
print(housing.target.shape)
print(housing.target.reshape(-1, 1).shape)

(20640,)
(20640, 1)


In [49]:
n_epochs = 10000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")

error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
gradients = 2/m * tf.matmul(tf.transpose(X), error)
traning_op = tf.assign(theta, theta - learning_rate * gradients)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 1000 == 0:
            print("Epoch", epoch, "MSE = ", mse.eval())
        sess.run(traning_op)
        
    best_theta = theta.eval()
best_theta

Epoch 0 MSE =  10.608322
Epoch 1000 MSE =  0.5266172
Epoch 2000 MSE =  0.5244344
Epoch 3000 MSE =  0.5243294
Epoch 4000 MSE =  0.524322
Epoch 5000 MSE =  0.52432114
Epoch 6000 MSE =  0.52432084
Epoch 7000 MSE =  0.52432084
Epoch 8000 MSE =  0.52432126
Epoch 9000 MSE =  0.524321


array([[ 2.0685523 ],
       [ 0.8296054 ],
       [ 0.11874886],
       [-0.26550296],
       [ 0.3056768 ],
       [-0.0045038 ],
       [-0.03932568],
       [-0.8999185 ],
       [-0.8705724 ]], dtype=float32)

## 9.6.2 자동 미분 사용
* 기호 미분(symbolic differentiation)을 이용해 자동으로 편미분 방정식을 구할 수 있으나 결과 코드의 효율이 나쁠 수 있다.
* 텐서플로는 자동 미분 기능을 통해 반복계산으로 오는 비효율을 해결, 효율적으로 그래디언트를 구한다.
* 자동으로 그래디언트를 구하는 방법들 중 텐서플로는 후진 모드 자동 미분 (reverse-mode autodiff) 를 사용한다.

In [56]:
def my_func(a, b):
    z = 0
    for i in range(100):
        z = a * np.cos(z + i) + z * np.sin(b - i)
    return z
my_func(0.2, 0.3)

-0.21253923284754916

In [60]:
tf.reset_default_graph()

a = tf.Variable(0.2, name="a")
b = tf.Variable(0.3, name="b")
z = tf.constant(0.0, name="z0")
for i in range(100):
    z = a * tf.cos(z + i) + z * tf.sin(b - i)

grads = tf.gradients(z, [a, b])
init = tf.global_variables_initializer()

with tf.Session() as sess:
    init.run()
    print(z.eval())
    print(sess.run(grads))

-0.21253741
[-1.1388494, 0.19671395]


## 9.6.3 옵티마이저 사용
* 텐서플로는 경사 하강법 옵티마이저를 포함해 여러가지 옵티마이저를 내장하고 있다.

In [64]:
tf.reset_default_graph()

n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")

optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("에포크", epoch, "MSE =", mse.eval())
        sess.run(training_op)
    
    best_theta = theta.eval()

print("best_theta:")
print(best_theta)

에포크 0 MSE = 2.7544262
에포크 100 MSE = 0.632222
에포크 200 MSE = 0.5727805
에포크 300 MSE = 0.5585007
에포크 400 MSE = 0.54907
에포크 500 MSE = 0.54228795
에포크 600 MSE = 0.5373789
에포크 700 MSE = 0.533822
에포크 800 MSE = 0.5312425
에포크 900 MSE = 0.5293704
best_theta:
[[ 2.06855249e+00]
 [ 7.74078071e-01]
 [ 1.31192386e-01]
 [-1.17845066e-01]
 [ 1.64778143e-01]
 [ 7.44078017e-04]
 [-3.91945094e-02]
 [-8.61356676e-01]
 [-8.23479772e-01]]


In [69]:
optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=0.5)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("에포크", epoch, "MSE =", mse.eval())
        sess.run(training_op)
    
    best_theta = theta.eval()

print("best_theta:")
print(best_theta)

에포크 0 MSE = 2.7544262
에포크 100 MSE = 0.5726946
에포크 200 MSE = 0.54910016
에포크 300 MSE = 0.5373619
에포크 400 MSE = 0.5312159
에포크 500 MSE = 0.5279875
에포크 600 MSE = 0.52628523
에포크 700 MSE = 0.525383
에포크 800 MSE = 0.5249017
에포크 900 MSE = 0.52464277
best_theta:
[[ 2.0685556 ]
 [ 0.814461  ]
 [ 0.12033767]
 [-0.22860533]
 [ 0.27152297]
 [-0.00372927]
 [-0.03913723]
 [-0.9025583 ]
 [-0.8710488 ]]


# 9.7 훈련 알고리즘에 데이터 주입
* 미니배치 경사 하강법을 구현하기 위해 플레이스홀더 사용
* 플레이스홀더(placeholder) : 실제로 아무 계산도 하지 않는 노드로 훈련간 데이터 전달을 위해 사용, 실행시 값을 지정하지 않으면 예외 발생
* <font color='red'>TODO</font> 미니배치가 무엇인지 스터디하기 > 메모리가 부족할때 데이터를 일부 로딩/처리를 배치로 반복하는 방법인듯함

In [73]:
tf.reset_default_graph()

A = tf.placeholder(tf.float32, shape=(None, 3))
B = A + 5
with tf.Session() as sess:
    B_val_1 = B.eval(feed_dict={A: [[1, 2, 3]]})
    B_val_2 = B.eval(feed_dict={A: [[4, 5, 6], [7, 8, 9]]})

print(B_val_1)
print(B_val_2)

[[6. 7. 8.]]
[[ 9. 10. 11.]
 [12. 13. 14.]]


In [77]:
tf.reset_default_graph()

n_epochs = 1000
learning_rate = 0.01

X = tf.placeholder(tf.float32, shape=(None, n + 1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")

theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

In [84]:
n_epochs = 10

batch_size = 100
n_batches = int(np.ceil(m / batch_size))

def fetch_batch(epoch, batch_index, batch_size):
    np.random.seed(epoch * n_batches + batch_index)  # not shown in the book
    indices = np.random.randint(m, size=batch_size)  # not shown
    X_batch = scaled_housing_data_plus_bias[indices] # not shown
    y_batch = housing.target.reshape(-1, 1)[indices] # not shown
    return X_batch, y_batch

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})

    best_theta = theta.eval()
best_theta

array([[ 2.0544672 ],
       [ 0.8297011 ],
       [ 0.10713524],
       [-0.3107482 ],
       [ 0.2456451 ],
       [-0.00222412],
       [-0.01061389],
       [-0.89121586],
       [-0.87524796]], dtype=float32)

# 9.8 모델 저장과 복원
* 계산된 모델 파라미터를 디스크에 저장하거나, 훈련 중 체크포인트 목적으로 저장 필요
* 그래프 구성의 끝에 Saver 노드 추가, 모델을 저장하고 싶을때 save() 메서드에 세션과 체크포인트 파일의 경로 전달

In [88]:
tf.reset_default_graph()

n_epochs = 1000                                                                       # 책에는 없습니다.
learning_rate = 0.01                                                                  # 책에는 없습니다.

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")            # 책에는 없습니다.
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")            # 책에는 없습니다.
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")                                      # 책에는 없습니다.
error = y_pred - y                                                                    # 책에는 없습니다.
mse = tf.reduce_mean(tf.square(error), name="mse")                                    # 책에는 없습니다.
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)            # 책에는 없습니다.
training_op = optimizer.minimize(mse)                                                 # 책에는 없습니다.

init = tf.global_variables_initializer()
saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("에포크", epoch, "MSE =", mse.eval())                                # 책에는 없습니다.
            save_path = saver.save(sess, "/tmp/my_model.ckpt")
        sess.run(training_op)
    
    best_theta = theta.eval()
    save_path = saver.save(sess, "/tmp/my_model_final.ckpt")
best_theta

에포크 0 MSE = 2.7544262
에포크 100 MSE = 0.632222
에포크 200 MSE = 0.5727805
에포크 300 MSE = 0.5585007
에포크 400 MSE = 0.54907
에포크 500 MSE = 0.54228795
에포크 600 MSE = 0.5373789
에포크 700 MSE = 0.533822
에포크 800 MSE = 0.5312425
에포크 900 MSE = 0.5293704


array([[ 2.06855249e+00],
       [ 7.74078071e-01],
       [ 1.31192386e-01],
       [-1.17845066e-01],
       [ 1.64778143e-01],
       [ 7.44078017e-04],
       [-3.91945094e-02],
       [-8.61356676e-01],
       [-8.23479772e-01]], dtype=float32)

In [89]:
with tf.Session() as sess:
    saver.restore(sess, "/tmp/my_model_final.ckpt")
    best_theta_restored = theta.eval() # 책에는 없습니다.
    
np.allclose(best_theta, best_theta_restored)

INFO:tensorflow:Restoring parameters from /tmp/my_model_final.ckpt


True

In [90]:
# theta 변수만 weights 라는 이름으로 저장
saver = tf.train.Saver({"weights": theta})

In [95]:
# 데이터 저장시 .meta 파일을 통해 그래프 정보도 함께 저장함
tf.reset_default_graph()

saver = tf.train.import_meta_graph("/tmp/my_model_final.ckpt.meta")  # 그래프 구조를 로드합니다.
theta = tf.get_default_graph().get_tensor_by_name("theta:0") # 책에는 없습니다.

with tf.Session() as sess:
    saver.restore(sess, "/tmp/my_model_final.ckpt")  # 그래프 상태를 로드합니다.
    best_theta_restored = theta.eval() # 책에는 없습니다.
best_theta_restored

INFO:tensorflow:Restoring parameters from /tmp/my_model_final.ckpt


array([[ 2.06855249e+00],
       [ 7.74078071e-01],
       [ 1.31192386e-01],
       [-1.17845066e-01],
       [ 1.64778143e-01],
       [ 7.44078017e-04],
       [-3.91945094e-02],
       [-8.61356676e-01],
       [-8.23479772e-01]], dtype=float32)

# 9.9 텐서보드로 그래프와 학습 곡선 시각화하기

In [97]:
from datetime import datetime
now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}".format(root_logdir, now)

print(logdir)

In [99]:
tf.reset_default_graph()

learning_rate = 0.01

X = tf.placeholder(tf.float32, shape=(None, n + 1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

mse_summary = tf.summary.scalar('MSE', mse)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

n_epochs = 10
batch_size = 100
n_batches = int(np.ceil(m / batch_size))

with tf.Session() as sess:                                                        # 책에는 없습니다.
    sess.run(init)                                                                # 책에는 없습니다.

    for epoch in range(n_epochs):                                                 # 책에는 없습니다.
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            if batch_index % 10 == 0:
                summary_str = mse_summary.eval(feed_dict={X: X_batch, y: y_batch})
                step = epoch * n_batches + batch_index
                file_writer.add_summary(summary_str, step)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})

    best_theta = theta.eval()                                                     # 책에는 없습니다.
    
file_writer.close()
best_theta

array([[ 2.070016  ],
       [ 0.8204561 ],
       [ 0.1173173 ],
       [-0.22739051],
       [ 0.3113402 ],
       [ 0.00353193],
       [-0.01126994],
       [-0.91643935],
       [-0.8795008 ]], dtype=float32)

# 9.10 이름 범위
* 신경망처럼 매우 복잡한 모델을 다룰 때는 계산 그래프가 수천 개의 노드로 복잡해질 수 있어 이름범위(name scope)를 이용해 노드들을 그룹으로 묶는게 필요하다.

In [103]:
tf.reset_default_graph()

from datetime import datetime
now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}".format(root_logdir, now)

print(logdir)

learning_rate = 0.01

X = tf.placeholder(tf.float32, shape=(None, n + 1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")

with tf.name_scope("loss") as scope:
    error = y_pred - y
    mse = tf.reduce_mean(tf.square(error), name = "mse")

optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

mse_summary = tf.summary.scalar('MSE', mse)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

n_epochs = 10
batch_size = 100
n_batches = int(np.ceil(m / batch_size))

with tf.Session() as sess:                                                        # 책에는 없습니다.
    sess.run(init)                                                                # 책에는 없습니다.

    for epoch in range(n_epochs):                                                 # 책에는 없습니다.
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            if batch_index % 10 == 0:
                summary_str = mse_summary.eval(feed_dict={X: X_batch, y: y_batch})
                step = epoch * n_batches + batch_index
                file_writer.add_summary(summary_str, step)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})

    best_theta = theta.eval()                                                     # 책에는 없습니다.
    
file_writer.close()
best_theta

tf_logs/run-20190120095937


array([[ 2.070016  ],
       [ 0.8204561 ],
       [ 0.1173173 ],
       [-0.22739051],
       [ 0.3113402 ],
       [ 0.00353193],
       [-0.01126994],
       [-0.91643935],
       [-0.8795008 ]], dtype=float32)

# 9.11 모듈화

In [104]:
# 두개의 ReLU 를 더하는 작업 수행 코드

tf.reset_default_graph()

n_features = 3
X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")

w1 = tf.Variable(tf.random_normal((n_features, 1)), name="weights1")
w2 = tf.Variable(tf.random_normal((n_features, 1)), name="weights2")
b1 = tf.Variable(0.0, name="bias1")
b2 = tf.Variable(0.0, name="bias2")

z1 = tf.add(tf.matmul(X, w1), b1, name="z1")
z2 = tf.add(tf.matmul(X, w2), b2, name="z2")

relu1 = tf.maximum(z1, 0., name="relu1")
relu2 = tf.maximum(z1, 0., name="relu2")  # Oops, cut&paste error! Did you spot it?

output = tf.add(relu1, relu2, name="output")

In [105]:
# 위의 코드의 반복적인 작성을 메서드로 변환한다.
tf.reset_default_graph()

def relu(X):
    w_shape = (int(X.get_shape()[1]), 1)
    w = tf.Variable(tf.random_normal(w_shape), name="weights")
    b = tf.Variable(0.0, name="bias")
    z = tf.add(tf.matmul(X, w), b, name="z")
    return tf.maximum(z, 0., name="relu")

n_features = 3
X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
relus = [relu(X) for i in range(5)]
output = tf.add_n(relus, name="output")

file_writer = tf.summary.FileWriter("logs/relu1", tf.get_default_graph())

In [106]:
# 동시에 name scope 까지 함께 하면 더 깔끔하게 정리 가능하다.
tf.reset_default_graph()

def relu(X):
    with tf.name_scope("relu"):
        w_shape = (int(X.get_shape()[1]), 1)                          # 책에는 없습니다.
        w = tf.Variable(tf.random_normal(w_shape), name="weights")    # 책에는 없습니다.
        b = tf.Variable(0.0, name="bias")                             # 책에는 없습니다.
        z = tf.add(tf.matmul(X, w), b, name="z")                      # 책에는 없습니다.
        return tf.maximum(z, 0., name="max")                          # 책에는 없습니다.
    
n_features = 3
X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
relus = [relu(X) for i in range(5)]
output = tf.add_n(relus, name="output")

file_writer = tf.summary.FileWriter("logs/relu2", tf.get_default_graph())
file_writer.close()

# 9.12 변수 공유

In [108]:
# 함수에 변수를 매개변수로 전달하는 방식
tf.reset_default_graph()

def relu(X, threshold):
    with tf.name_scope("relu"):
        w_shape = (int(X.get_shape()[1]), 1)                        # 책에는 없습니다.
        w = tf.Variable(tf.random_normal(w_shape), name="weights")  # 책에는 없습니다.
        b = tf.Variable(0.0, name="bias")                           # 책에는 없습니다.
        z = tf.add(tf.matmul(X, w), b, name="z")                    # 책에는 없습니다.
        return tf.maximum(z, threshold, name="max")

threshold = tf.Variable(0.0, name="threshold")
X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
relus = [relu(X, threshold) for i in range(5)]
output = tf.add_n(relus, name="output")

In [ ]:
# 모델에 모든 변수를 담은 파이썬 딕셔너리를 만들고 함수마다 이를 전달하는 방식
tf.reset_default_graph()

params["threshold"] = tf.Variable(0.0, name="threshold")

def relu(X, params):
    with tf.name_scope("relu"):
        w_shape = (int(X.get_shape()[1]), 1)                        # 책에는 없습니다.
        w = tf.Variable(tf.random_normal(w_shape), name="weights")  # 책에는 없습니다.
        b = tf.Variable(0.0, name="bias")                           # 책에는 없습니다.
        z = tf.add(tf.matmul(X, w), b, name="z")                    # 책에는 없습니다.
        return tf.maximum(z, params["threshold"], name="max")

X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
relus = [relu(X, params) for i in range(5)]
output = tf.add_n(relus, name="output")

In [111]:
# 공유변수 지정
tf.reset_default_graph()

def relu(X):
    with tf.name_scope("relu"):
        if not hasattr(relu, "threshold"):
            relu.threshold = tf.Variable(0.0, name="threshold")
        w_shape = int(X.get_shape()[1]), 1                          # 책에는 없습니다.
        w = tf.Variable(tf.random_normal(w_shape), name="weights")  # 책에는 없습니다.
        b = tf.Variable(0.0, name="bias")                           # 책에는 없습니다.
        z = tf.add(tf.matmul(X, w), b, name="z")                    # 책에는 없습니다.
        return tf.maximum(z, relu.threshold, name="max")

In [120]:
# 텐서플로에서 많이 사용하는 방식
tf.reset_default_graph()

# threshold 를 초기값으로 생성, 이미 변수가 존재하는 경우 오류 발생
with tf.variable_scope("relu"):
    threshold = tf.get_variable("threshold", shape=(),
                               initializer=tf.constant_initializer(0.0))

# 이미 초기화된 변수 이용, 초기화되지 않은 경우 오류 발생
with tf.variable_scope("relu", reuse=True):
    threshold = tf.get_variable("threshold")

In [136]:


def relu(X):
    with tf.variable_scope("relu", reuse=True):
        threshold = tf.get_variable("threshold")
        w_shape = int(X.get_shape()[1]), 1                          # 책에는 없습니다.
        w = tf.Variable(tf.random_normal(w_shape), name="weights")  # 책에는 없습니다.
        b = tf.Variable(0.0, name="bias")                           # 책에는 없습니다.
        z = tf.add(tf.matmul(X, w), b, name="z")                    # 책에는 없습니다.
        return tf.maximum(z, threshold, name="max")

X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
with tf.variable_scope("relu"):
    threshold = tf.get_variable("threshold", shape=(),
                                initializer=tf.constant_initializer(0.0))
relus = [relu(X) for relu_index in range(5)]
output = tf.add_n(relus, name="output")

file_writer = tf.summary.FileWriter("logs/relu6", tf.get_default_graph())
file_writer.close()

In [137]:
tf.reset_default_graph()

def relu(X):
    threshold = tf.get_variable("threshold", shape=(),
                                initializer=tf.constant_initializer(0.0))
    w_shape = (int(X.get_shape()[1]), 1)                        # 책에는 없습니다.
    w = tf.Variable(tf.random_normal(w_shape), name="weights")  # 책에는 없습니다.
    b = tf.Variable(0.0, name="bias")                           # 책에는 없습니다.
    z = tf.add(tf.matmul(X, w), b, name="z")                    # 책에는 없습니다.
    return tf.maximum(z, threshold, name="max")

X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
relus = []
for relu_index in range(5):
    with tf.variable_scope("relu", reuse=(relu_index >= 1)) as scope:
        relus.append(relu(X))
output = tf.add_n(relus, name="output")

file_writer = tf.summary.FileWriter("logs/relu9", tf.get_default_graph())
file_writer.close()